---

<div style="display: flex; align-items: center; justify-content: space-between;">
  <img src="../../.images/PNAV-logo.png" alt="Logo del PNAV" style="width: auto; max-height: 100px;">
  <img src="../../.images/MITECO-logo_background.png" alt="Logo del MITECO" style="width: auto; max-height: 100px;">
</div>

---

# Generación de un dataset de preguntas y respuestas para hacer fine-tuning de LLMs

A continuación, se mostrará el proceso de cómo generar un dataset de preguntas y respuestas básico para hacer fine-tuning de un LLM.

⚠️ Este material es un notebook de demostración **orientativo** sobre la metodología que se debe seguir, en ningún caso se debe replicar el desarrollo del caso de uso.

## 1. Importación de módulos y librerías
Se importan los módulos y librerías necesarios para ejecutar el código.

In [1]:
import os
import requests
import pandas as pd

## 2. Petición de los datos mediante API

In [2]:
# URL de la API
url = "https://iepnb.gob.es:443/api/especie/v_estadolegal?idtaxon=gt.%200"

# Realizar la petición GET
response = requests.get(url)

# Verificar si la solicitud fue exitosa (código 200)
if response.status_code == 200:
    # Procesar los datos (por ejemplo, si la respuesta es JSON)
    data = response.json()
    df = pd.DataFrame(data)
    df = df.dropna(subset=['nombre_aceptado']).reset_index()
else:
    print(f"Error: {response.status_code}")

## 3. Generación del dataset de Q&A

In [3]:
data = []
output_path = 'output'
os.makedirs(output_path, exist_ok=True)

for index, row in df.iterrows():
    # Formar la pregunta básica
    pregunta = f"¿Cuál es el estado legal de la especie {row['nombre_aceptado']}"
    
    # Si el ámbito es Regional, incluir la comunidad autónoma en la pregunta
    if row['ambito'] == 'Regional' and 'ccaa' in row and pd.notnull(row['ccaa']):
        pregunta += f" en la comunidad autónoma de {row['ccaa']}?"
    elif row['ambito'] == 'Internacional':
        pregunta += f" a nivel internacional?"
    elif row['ambito'] == 'Nacional':
        pregunta += f" a nivel nacional?"
    else:
        pregunta += f"?"

    # Generar la respuesta con el estado legal y la norma
    respuesta = f"La especie {row['nombre_aceptado']} se encuentra en estado de {row['estadolegal'].lower()}"
    
    # Incluir el ámbito en la respuesta
    respuesta += f" a nivel {row['ambito'].lower()}"
    
    # Si el ámbito es Regional, incluir la comunidad autónoma
    if row['ambito'] == 'Regional' and 'ccaa' in row and pd.notnull(row['ccaa']):
        respuesta += f" en la comunidad autónoma de {row['ccaa']}."
    elif row['ambito'] == 'Internacional':
        respuesta += f" a nivel internacional."
    elif row['ambito'] == 'Nacional':
        respuesta += f" a nivel nacional."

    # Añadir la pregunta y la respuesta al dataset
    data.append({'pregunta': pregunta, 'respuesta': respuesta})

# Crear el DataFrame de preguntas y respuestas
df_preguntas_respuestas = pd.DataFrame(data)

#df_preguntas_respuestas.to_csv(os.path.join(output_path, f'qa_dataset.csv'))

## 4. Se adapta al formato de entrenamiento del modelo

In [5]:
import pandas as pd

# Convertir el dataframe en el formato deseado
def convertir_a_dataset(df):
    dataset = []
    for _, row in df.iterrows():
        pregunta = row['pregunta']
        respuesta = row['respuesta']
        text = (
            "Below is an instruction that describes a task, paired with an input that provides further context. "
            "Write a response that appropriately completes the request.\n\n"
            f"### Instruction:\n{pregunta}\n\n"
            "### Input:\n\n"
            "### Response:\n"
            f"{respuesta}<|im_end|>"
        )
        dataset.append({
            'output': respuesta,
            'input': '',
            'instruction': pregunta,
            'text': text
        })
    return dataset

# Aplicar la función al dataframe
dataset = convertir_a_dataset(df_preguntas_respuestas)